In [43]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") 
include("$srcpath/bp_stab.jl") 
include("$srcpath/post_infer.jl") 
include("$srcpath/observables.jl")

avg_ninf (generic function with 1 method)

In [84]:
λp = 1.0# planted infection rate
λi = λp #inferred infection rate

T = 8 # discrete time: number of time-steps

γp = 0.01 #0.019 # planted autoinfection probability
γi = γp # inferred autoinfection probability

N = 5000; #population size
@show 2/sqrt(N)

dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 

degree_dist = Dirac(3) #the distribution of the degree in the graph. Dirac means random regular.

2 / sqrt(N) = 0.0282842712474619


Dirac{Int64}(value=3)

In [85]:
#M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [86]:
#@time sweep!(M_zero)

In [87]:
#F_zero,it_zero = pop_dynamics(M_zero, tot_iterations = 20);

In [88]:
#l=rand(1:N)
#M_zero.belief[:,:,l]

In [89]:
#marg_zero = M_zero.belief |> real;
#marg2D_zero = reshape(sum(marg_zero,dims=3) ./ N, T+2,T+2)
#ensAUC_zero = avgAUC(marg_zero,M_zero.obs_list,count_obs=true);

In [95]:
include("$srcpath/post_infer.jl") 

δλ (generic function with 1 method)

In [96]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [97]:
#@time sweep_stab!(M, 3, 4)

In [98]:
pop_dynamics_stab(M, tot_iterations = 50);

#1.iter 2.conv_crit 3.F 4. Δ
1	Inf	-1	
2	0.1957003936790862	-1	
3	0.18821206020500725	-1	
4	0.22035799252401134	-1	
5	0.13549398175116778	-1	
6	0.059839585477716284	-1	
7	0.058157211701377776	-1	
8	0.07293210952603246	-1	
9	0.22665866052239458	-1	
10	0.5088183550996064	-1	
11	0.4854550873741399	-1	
12	0.011156640244035287	-1	
13	0.00015034653648449137	-1	
14	0.27113270227960484	-1	
15	0.3578104191845952	-1	
16	0.17380001292193548	-1	
17	0.4351068253345183	-1	
18	0.5472922597966665	-1	
19	0.36329635298906915	-1	
20	0.08198443161226088	-1	
21	0.18232840650168392	-1	
22	0.08667068894458652	-1	
23	0.4140022506073251	-1	
24	1.010810602492601	-1	
25	0.7846761721664564	-1	
26	0.31104095048219804	6.927173872185184e7	
27	0.16464874133633423	3.739883189060415e9	
28	0.2984144542526934	2.3618472465944546e10	
29	1.0771990610297806	3.420614429915289e11	
30	0.950551683520878	6.132164512386117e12	
31	0.17956793500408708	1.5372920416925216e24	
32	0.18522713025111592	4.771996791909052e24	
33	0.145790107

In [ ]:
marg = M.belief |> real;
marg2D = reshape(sum(marg,dims=3) ./ N, T+2,T+2)
ensAUC = avgAUC(marg,M.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
sum(abs.(marg2D.-marg2D_zero))/length(marg2D)

In [ ]:
scatter(ensAUC)
scatter!(ensAUC_zero)

In [ ]:
pheat = heatmap((marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred",title="Average 2D marginal")
pheat_stab = heatmap((marg2D_zero.-marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred")
plot(pheat,pheat_stab)

In [ ]:
scatter(F)
scatter!(F_zero)
hline!([0])